In [45]:
import minsearch

In [46]:
import json


In [47]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [48]:
# docs_raw

In [49]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [50]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [51]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'])



In [52]:
index.fit(documents)

In [53]:
q = 'the course has already started, can I still enroll?'

In [55]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [56]:
from google import genai
from google.genai import types

In [ ]:

# Only run this block for Gemini Developer API
client = genai.Client(api_key=GOOGLE_API_KEY)

In [58]:
# response = client.models.generate_content(
#     model='gemini-2.5-pro', 
#     contents=q
# )
# print(response.text)

In [63]:
response = client.models.generate_content(
    model='gemini-2.5-pro', 
    contents=[q, prompt]
    
)
print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects.


In [64]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5} #'text': 1.0,, 'course': 0.1}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5)

    return results


#### prompt template

In [72]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT provided. Only use the facts from the CONTEXT to answer the QUESTION. If the CONTEXT does not contain enough information to answer the QUESTION.
    If the CONTEXT doesnt contain the answer, output NONE

    QUESTON: {question}
    CONTEXT:
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    # print(context)

    prompt = prompt_template.format(
        question=q,
        context=context
        ).strip()

    return prompt


In [140]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Example: get a variable
google_api_key = os.getenv('GOOGLE_API_KEY')


In [ ]:
def llm(query, prompt):
    client = genai.Client(api_key=google_api_key)

    response = client.models.generate_content(
        model='gemini-2.5-pro', 
        contents=[query, prompt]
    )
    
    return response.text.strip()

In [78]:
query = 'the course has already started, can I still enroll?'

def rag(query):
    """
    RAG: Retrieve, Augment, Generate
    """

    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(query, prompt) 

    return answer

#### token count

In [82]:
# Access the usage metadata from the response
usage_data = response.usage_metadata

print("--- Token Usage ---")
print(f"Prompt Tokens (Input): {usage_data.prompt_token_count}")
print(f"Candidates Tokens (Output): {usage_data.candidates_token_count}")
print(f"Total Tokens: {usage_data.total_token_count}")

--- Token Usage ---
Prompt Tokens (Input): 576
Candidates Tokens (Output): 38
Total Tokens: 1001


#### elasticsearch

In [91]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [92]:
from elasticsearch import Elasticsearch

In [98]:
from opensearchpy import OpenSearch # <-- Change here

In [ ]:
# es_client = Elasticsearch('http://localhost:9200')

# DataTalksClub's Zoomcamp often uses OpenSearch, which is a fork of Elasticsearch 7.10.2. To an elasticsearch-py client, it looks like an old Elasticsearch 7 server.
es_client = OpenSearch('http://localhost:9200') # <-- Change here

In [100]:
es_client.info()

{'name': '9c46976db000',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'F5--G66NT3y_kL2XQpDb5g',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [112]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [102]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [109]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        body=doc
    )

  0%|          | 0/948 [00:00<?, ?it/s]

In [126]:
def f_elasticsearch(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # weighting
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [117]:
query = 'the course has already started, can I still enroll?'

In [127]:
f_elasticsearch(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [125]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [130]:
def rag_elastic(query):
    """
    RAG: Retrieve, Augment, Generate
    """

    search_results = f_elasticsearch(query)
    prompt = build_prompt(query, search_results)
    answer = llm(query, prompt) 

    return answer

In [131]:
rag_elastic(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."